In [ ]:
from pycoreimage.pyci import cimg
from pycoreimage.pyci import show
from pycoreimage.pyci import color
# load an image from file
originImg = cimg.fromFile('./resources/Flowers_1.jpg')
#print(type(originImg))
#print(originImg)
bar = originImg.render() #图片缓存 numpy 表示
show(originImg.render(), title='{}X{}'.format(*originImg.size))

### Core Image filters

In [ ]:
# apply a filter
gaussImg = originImg.gaussianBlur(radius=30)
show([originImg, gaussImg])

In [ ]:
 fs = cimg.filters() # build-in filters
 for i, f in enumerate(cimg.filters()):
    print('{:3d} {}'.format(i,f))

In [ ]:
cimg.inputs('CIZoomBlur') # filters inputs

In [ ]:
zoomBImg = originImg.zoomBlur(center=[originImg.size[0] / 2,originImg.size[1] / 2],Amount=5)
show([originImg,zoomBImg])


### Core Image Generator

In [ ]:
# 生成二维码
qr = cimg.fromGenerator('CIQRCodeGenerator', message='kiss me')
show([qr], interpolation='nearest')

In [ ]:
wwdc = cimg.fromGenerator('CITextImageGenerator',
                          text='WWDC\n 2019',
                          fontName='SFHello-Bold',
                          fontSize=400,
                          scaleFactor=1).colorInvert()
show(wwdc.scale(0.5,0.5),title="wwdc")

In [ ]:
bridge = cimg.fromFile('./resources/GGBridge_2.jpg')
distortion = bridge.vortexDistortion(inputCenter=(bridge.size[0]/2, bridge.size[1]/2), 
                                   radius=bridge.size[0],
                                   angle=4000)
show([bridge,distortion])

In [ ]:
ary =  distortion.render()
import numpy as np
print('array type', type(ary), ary.shape, ary.dtype)
print('array stats = ', ary.min(), np.percentile(ary, 50), ary.max())

### Numpy to CoreImge

In [ ]:
noise = np.random.rand(512,512,3)
noise[noise < 0.75] = 0
show(noise)

In [ ]:
noiseImg = cimg(noise)
print(type(noiseImg))
print(noiseImg.ciimage)

In [ ]:
#print([10]*2)
fimg1 = noiseImg.discBlur(radius = 10)
fimg2 = fimg1.lightTunnel(center = [noiseImg.size[0]/2]*2, radius = noiseImg.size[0] / 3)
fimg3 = fimg2.colorControls(contrast = 1.2, saturation = 1.3)
fimg4 = fimg3.exposureAdjust(EV = 2).gammaAdjust(power = 2)
show(fimg1, title = 'disc blur', at = 141)
show(fimg2, title = 'light tunnel', at = 142)
show(fimg3, title = 'color control', at = 143)
show(fimg4, title = 'exposure Adjust', at = 144)
show(fimg4, title = 'Numpy to Core Image')

### Custom kernel

In [ ]:
# five tap Laplacian 
#  0  -1   0
# -1  4  -1
#  0  -1   0
source = """ kernel vec4 sharpen(samplers, float amount) {
    vec2 p = destCoord();
    vec4 a = sample(s, samplerTransform(s, p + vec2(-1.0,  0.0)));
    vec4 b = sample(s, samplerTransform(s, p + vec2( 0.0,  1.0)));
    vec4 c = sample(s, samplerTransform(s, p));
    vec4 d = sample(s, samplerTransform(s, p + vec2( 1.0,  0.0)));
    vec3 e = sample(s, samplerTransform(s, p + vec2( 0.0, -1.0)));
    vec4 L = 4.0 * c - a - b - d  - e;
    return vec4(c.rgb + amount * L.rgb, c.a);
} """

sample = cimg.fromFile('resources/SampleImage.jpg');
result = sample.applyKernel(source,
                            2,
                            extent = (255, 500, 256, 256),
                            roi = lambda index, r: inset(r, -1, -1))
show([sample, result])

### Putting it all together

In [ ]:
aircraft = cimg.fromFile('resources/aircraft.png').scale(0.5,0.5)
show(aircraft)

In [ ]:
composite = np.zeros((int(aircraft.size[1]), int(aircraft.size[0]), 3),dtype=float)
#print(composite)
rows = aircraft.size[1]

def addBand(i, name, args):
    # apply filter via explicit name and args
    band = aircraft.applyFilter(name, **args)
    #print(band.render())
    # create a label with filter name
    label = cimg.fromGenerator('CITextImageGenerator',
                               text = name,
                               fontName = 'SFHello',
                               fontSize = 80,
                               scaleFactor=1)
    label = label.colorInvert().whitePointAdjust(color = color(1, 0, 0, 1))
    
    # layer text over the image
    lo, hi = i * rows, (i + 1) * rows
    label = label.translate(0.01 * composite.shape[1], composite.shape[0] - hi + (hi - lo) / 2)
    band = label.over(band)
    
    # slice the CIImage here: render only happens in that band 
    print(band.render().shape,band.render().dtpye)
    composite[lo:hi, ...] = band.render()[lo:hi,...]

In [ ]:
addBand(0, 'photoEffectNoir', {})
#addBand(1, 'morphologyMinimum', {'radius': 10})
#addBand(2, 'gaussianBlur', {'radius': 10})
#addBand(3, 'vibrance', {'amount': 8})
#addBand(4, 'pixellate', {'center': (0,0), 'scale': 25})
#show(composite)